In [4]:
# pacifica_client.py
import os, time, uuid, pathlib
import requests, base58
from dotenv import load_dotenv
from solders.keypair import Keypair
from common.constants import REST_URL
from common.utils import sign_message

# 明示的に keys.env を読む（ファイル名が .env でないため）
load_dotenv("keys.env")

OWNER_PRIVATE_KEY = os.getenv("OWNER_PRIVATE_KEY_BASE58")  # 本体ウォレット
API_KEY           = os.getenv("PACIFICA_API_KEY")
AGENT_KEY_PATH    = os.getenv("AGENT_KEY_PATH", "agent_wallet_base58.key")

BIND_URL   = f"{REST_URL}/agent/bind"
MKT_URL    = f"{REST_URL}/orders/create_market"
LIMIT_URL  = f"{REST_URL}/orders/create"          # ドキュメントの create（TP/SL対応）に合わせる場合
CANCEL_ALL = f"{REST_URL}/orders/cancel_all"      # 例：後で使う用

def _headers():
    h = {"Content-Type": "application/json"}
    if API_KEY:  # エンドポイントにより必須の場合あり
        h["X-API-Key"] = API_KEY
    return h

def load_owner() -> Keypair:
    if not OWNER_PRIVATE_KEY:
        raise RuntimeError("OWNER_PRIVATE_KEY_BASE58 が keys.env にありません。")
    return Keypair.from_base58_string(OWNER_PRIVATE_KEY)

def load_or_create_agent(path: str = AGENT_KEY_PATH) -> Keypair:
    p = pathlib.Path(path)
    if p.exists():
        return Keypair.from_base58_string(p.read_text().strip())
    k = Keypair()
    p.write_text(base58.b58encode(k.to_bytes()).decode())
    try: os.chmod(p, 0o600)
    except: pass
    return k

def bind_agent_wallet(owner_kp: Keypair, agent_pubkey: str):
    ts = int(time.time() * 1_000)
    header = {"timestamp": ts, "expiry_window": 5_000, "type": "bind_agent_wallet"}
    payload = {"agent_wallet": agent_pubkey}
    _, sig = sign_message(header, payload, owner_kp)

    body = {
        "account": str(owner_kp.pubkey()),
        "signature": sig,
        "timestamp": ts,
        "expiry_window": header["expiry_window"],
        "agent_wallet": agent_pubkey,
    }
    r = requests.post(BIND_URL, json=body, headers=_headers(), timeout=15)
    return r

def create_market_order(owner_pub: str, agent_kp: Keypair, symbol: str, side: str,
                        amount: str, reduce_only: bool=False, slippage_percent: str="0.5",
                        client_order_id: str | None=None):
    ts = int(time.time() * 1_000)
    if client_order_id is None:
        client_order_id = str(uuid.uuid4())
    header = {"timestamp": ts, "expiry_window": 5_000, "type": "create_market_order"}
    payload = {
        "symbol": symbol,
        "reduce_only": reduce_only,
        "amount": amount,
        "side": side,  # "bid" or "ask"
        "slippage_percent": slippage_percent,
        "client_order_id": client_order_id,
    }
    _, sig = sign_message(header, payload, agent_kp)
    body = {
        "account": owner_pub,
        "agent_wallet": str(agent_kp.pubkey()),
        "signature": sig,
        "timestamp": ts,
        "expiry_window": header["expiry_window"],
        **payload,
    }
    return requests.post(MKT_URL, json=body, headers=_headers(), timeout=15)

def create_limit_order(owner_pub: str, agent_kp: Keypair, symbol: str, side: str,
                       amount: str, price: str, reduce_only: bool=False,
                       client_order_id: str | None=None,
                       take_profit: str | None=None, stop_loss: str | None=None):
    """ドキュメントの /orders/create 用（任意のTP/SL付き）"""
    ts = int(time.time() * 1_000)
    if client_order_id is None:
        client_order_id = str(uuid.uuid4())
    header = {"timestamp": ts, "expiry_window": 5_000, "type": "create_order"}  # ← docs
    payload = {
        "symbol": symbol,
        "side": side,             # "bid"/"ask"
        "amount": amount,
        "price": price,
        "reduce_only": reduce_only,
        "client_order_id": client_order_id,
    }
    if take_profit is not None:
        payload["take_profit"] = take_profit
    if stop_loss is not None:
        payload["stop_loss"] = stop_loss

    _, sig = sign_message(header, payload, agent_kp)
    body = {
        "account": owner_pub,
        "agent_wallet": str(agent_kp.pubkey()),
        "signature": sig,
        "timestamp": ts,
        "expiry_window": header["expiry_window"],
        **payload,
    }
    return requests.post(LIMIT_URL, json=body, headers=_headers(), timeout=15)


In [7]:
def main():
    owner = load_owner()
    agent = load_or_create_agent()
    owner_pub = str(owner.pubkey())

    # 1) 初回/冪等 bind
    r = bind_agent_wallet(owner, str(agent.pubkey()))
    print("[bind]", r.status_code, r.text)

    # 2) 成行の例（SOLを0.1枚買い）
    r = create_market_order(owner_pub, agent, symbol="SOL", side="ask", amount="0.05")
    print("[mkt ]", r.status_code, r.text)

    # 3) 指値の例（任意：TP/SL付き）
    # r = create_limit_order(owner_pub, agent, symbol="SOL", side="ask", amount="0.1",
    #                        price="210.00", take_profit="215", stop_loss="205")
    # print("[lmt ]", r.status_code, r.text)

if __name__ == "__main__":
    main()


[bind] 200 {"success":true,"data":null,"error":null,"code":null}
[mkt ] 200 {"success":true,"data":{"order_id":327782065},"error":null,"code":null}
